# 🧪 Test JEPA3D Encoder

This notebook shows how to import and run your `Encoder` wrapper from  
`src/models/encoders/jepa3d_backbone.py`.


In [1]:
import torch
from models.encoders.jepa3d_backbone import Encoder
from ext.jepa3d.models.encoder_3djepa import Encoder3DJEPA

prbint("✅ Both imports resolve!")


ModuleNotFoundError: No module named 'torch_scatter'

In [2]:
# In your notebook
import torch
from models.encoders.jepa3d_backbone import Encoder
from ext.jepa3d.models.encoder_3djepa import Encoder3DJEPA

print("✅ Imports work!")


ModuleNotFoundError: No module named 'models.point_transformer_v3'